### Preprocessing of Functional Brain Network Connectivity Matrices

In [ ]:

import os
import glob
import scipy.io
import json
import numpy as np


def safe_minmax_scale(matrix, epsilon=1e-5, delta=1e-5):
    """
    Perform min-max scaling on non-zero elements of the matrix with safety bounds
    
    Args:
        matrix: Input connectivity matrix
        epsilon: Small value to prevent values from reaching exactly 1
        delta: Small value to prevent values from reaching exactly 0
    
    Returns:
        Scaled matrix where non-zero elements are normalized to [delta, 1-epsilon]
    """
    # Create mask to identify non-zero elements
    mask = matrix != 0
    values = matrix[mask]
    
    # Handle case where all non-zero values are the same
    if len(values) == 0:
        return matrix.copy()
        
    min_val = np.min(values)
    max_val = np.max(values)
    
    if max_val == min_val:
        return matrix.copy()
    
    # Apply min-max scaling only to non-zero elements
    scaled = matrix.copy()
    scaled[mask] = delta + ((matrix[mask] - min_val) / (max_val - min_val)) * (1 - epsilon - delta)
    return scaled

def masked_softmax(matrix):
    """
    Apply row-wise softmax only to non-zero elements with numerical stability
    
    Args:
        matrix: Input connectivity matrix
        
    Returns:
        Matrix with softmax applied row-wise to non-zero elements
    """
    rowwise_result = np.zeros_like(matrix)
    
    # Process each row individually
    for i in range(matrix.shape[0]):
        row = matrix[i]
        mask = row != 0
        values = row[mask]
        
        # Skip rows with no non-zero elements
        if len(values) == 0:
            continue
            
        # Numerical stability: subtract max value to prevent overflow
        shifted = values - np.max(values)
        exps = np.exp(shifted)
        softmaxed = exps / np.sum(exps)
        
        # Apply softmax only to non-zero positions
        rowwise_result[i, mask] = softmaxed
        
    return rowwise_result

def process_mat_files(
    folder_path,
    output_file,
    save_to_file=False
):
    """
    Process all functional brain network connectivity matrices in a folder
    
    Args:
        folder_path: Path to folder containing multiple .mat files with connectivity matrices
        output_file: Path for output JSON file
        save_to_file: Whether to save results to file
        
    Returns:
        Dictionary containing processed adjacency matrices
    """
    # Find all .mat files in the folder
    mat_files = glob.glob(os.path.join(folder_path, '*.mat'))
    
    # Extract matrix names from filenames
    matrix_names = [os.path.splitext(os.path.basename(file))[0].split('_')[0] for file in mat_files]
    adjacency_matrices = {}

    # Process each .mat file
    for file_path, matrix_name in zip(mat_files, matrix_names):
        # Load .mat file data
        data = scipy.io.loadmat(file_path)
        
        # Check if the expected matrix name exists in the .mat file
        if matrix_name not in data:
            print(f"Warning: Key '{matrix_name}' not found in {file_path}, skipping this file.")
            continue

        # Extract and convert matrix to float
        matrix = data[matrix_name].astype(float)
        # Keep only positive connections (set negative values to zero)
        matrix[matrix < 0] = 0
        # Apply row-wise softmax to non-zero elements
        matrix = masked_softmax(matrix) 
        # Apply min-max scaling to non-zero elements
        matrix = safe_minmax_scale(matrix)

        # Store processed matrix in dictionary
        adjacency_matrices[matrix_name] = matrix.tolist()

    # Save results to JSON file if requested
    if save_to_file:
        with open(output_file, 'w') as json_file:
            json.dump(adjacency_matrices, json_file, indent=4)
        print(f"Results saved to {output_file}")

    return adjacency_matrices

In [ ]:
# Example usage
folder_path = 'your_folder_path'
output_file = 'your_output_file_path.json'
process_mat_files(
    folder_path,
    output_file,
    save_to_file=True
)

### Computation of Graph Metrics for Functional Brain Networks

In [ ]:
import json
import networkx as nx
import numpy as np
import community  
import random

def compute_graph_metrics(brain_graph_results_path, file_path, save_to_file=False):
    """
    Compute graph metrics (divided into connection strength graph and path length graph categories):
    - Weighted global clustering coefficient (Barrat formula)
    - Weighted modularity (Louvain algorithm)
    - Weighted average shortest path
    - Global efficiency
    - Degree distribution statistics

    Parameters:
      brain_graph_results_path: Path to JSON file containing aggregated matrices
      file_path: Path to save graph metrics results
      save_to_file: Whether to save results to file
    Returns:
      graph_metric: Dictionary of graph metrics for each model (including both connection strength and path length graph metrics)
    """
    # 1. Read JSON file
    with open(brain_graph_results_path, 'r') as f:
        aggregation_results = json.load(f)
    
    graph_metric = {}  # Store graph metrics for each model

    # 2. Iterate through each model
    for model_name, matrix in aggregation_results.items():
        # Convert list to NumPy array
        matrix = np.array(matrix)
        n = matrix.shape[0]
        
        # 🔹 Connection Strength Graph
        conn_matrix = (matrix + matrix.T) / 2  # Symmetrize
        G_conn = nx.from_numpy_array(conn_matrix)

        # 🔹 Path Length Graph
        distance_matrix = np.where(matrix != 0, 1 - np.abs(matrix), np.inf)
        distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Symmetrize
        G_dist = nx.from_numpy_array(distance_matrix)

        # 3. Calculate graph metrics
        metrics = {}

        # Structural metrics (based on connection strength graph G_conn)
        try:
            metrics['clustering'] = nx.average_clustering(G_conn, weight='weight')
        except Exception:
            metrics['clustering'] = None

        try:
            partition = community.best_partition(G_conn, weight='weight', random_state=0)
            metrics['modularity'] = community.modularity(partition, G_conn, weight='weight')
        except Exception:
            metrics['modularity'] = None

        try:
            degrees = [deg for _, deg in G_conn.degree(weight='weight')]
            metrics['average_degree'] = np.mean(degrees)
            metrics['degree_std'] = np.std(degrees)
        except Exception:
            metrics['average_degree'] = None
            metrics['degree_std'] = None

        # Path-based metrics (based on path length graph G_dist)
        try:
            if nx.is_connected(G_dist):
                metrics['average_shortest_path'] = nx.average_shortest_path_length(G_dist, weight='weight')
            else:
                metrics['average_shortest_path'] = float('inf')
        except Exception:
            metrics['average_shortest_path'] = None

        try:
            length = dict(nx.all_pairs_dijkstra_path_length(G_dist, weight='weight'))
            total_eff = sum(1 / d for src in length for tgt, d in length[src].items() if src != tgt)
            metrics['global_efficiency'] = total_eff / (n * (n - 1))
        except Exception:
            metrics['global_efficiency'] = None

        
        # 4. Store results for this model
        graph_metric[model_name] = metrics

    # 5. Save results to JSON file (optional)
    if save_to_file:
        with open(file_path, 'w') as f:
            json.dump(graph_metric, f, indent=4)
        print(f"Graph metrics saved to {file_path}")
    
    return graph_metric

In [ ]:
# Example usage
graph_measure = compute_graph_metrics('your_brain_graph_results_path.json', 'your_output_file_path.json', save_to_file=True)
##'your_brain_graph_results_path.json' should be the 'your_output_file_path.json' from the previous section